In [1]:
from transformers import BertTokenizerFast,BertTokenizer
from datasets import DatasetDict,Dataset,concatenate_datasets

from corpus import Loader,Type
from utils import dataset_util
from utils import Labeler

In [2]:
def make_ready_for_ds(data,corpus_type,labeler,tokenizer,chunk_size=512):
    chars, labels = labeler.label_text(data,corpus_type)
    input_ids_list, labels_list =[],[]
    for i in range(len(chars)):
        tokenized_sentence = [101] +[tokenizer.encode(char)[1] for char in chars[0]]+[102]  
        labels[0].insert(0,0) # Adding CLS token label at the beginning of each sequence
        labels[0].append(0)
        input_ids_list+=tokenized_sentence
        labels_list+=labels[0]
        chars.pop(0)
        labels.pop(0)
    input_ids_list,attention_mask_list,labels_list=dataset_util.chunk_pad_tokens(input_ids_list,labels_list,chunk_size,padd=True)
    # print(len(input_ids_list),len(attention_mask_list),len(labels_list))
    # for i in range(max(len(input_ids_list),len(attention_mask_list),len(labels_list))):
    #     print(len(input_ids_list[i]),len(attention_mask_list[i]),len(labels_list[i]))
    return input_ids_list,attention_mask_list,labels_list


def build_dataset(input_ids_list,attention_mask_list,labels_list,dataset_path):
    data_dict = {
        "input_ids": input_ids_list,
        "labels": labels_list,
        "attention_mask": attention_mask_list,
    }
    # Create a Hugging Face Dataset from the dictionary
    dataset = Dataset.from_dict(data_dict)
    # Shuffle the dataset
    dataset = dataset.shuffle(seed=42)
    # Split the dataset into training, validation, and test sets
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))    
    train_dataset = dataset.select(range(train_size))
    val_dataset = dataset.select(range(train_size, train_size + val_size))
    test_dataset = dataset.select(range(train_size + val_size, len(dataset)))   
    # Combine the datasets into a DatasetDict
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })
    dataset_dict._metadata = {"author": "Matin Ebrahimkhani"}
    dataset_dict.save_to_disk(dataset_path)
    return dataset_dict


def add_data_to_dataset(input_ids_list, attention_mask_list, labels_list,dataset_path,save_path):
    # Load the dataset from disk
    original_ds = DatasetDict.load_from_disk(dataset_path)
    # Create a dictionary from the input lists
    data_dict = {
        "input_ids": input_ids_list,
        "labels": labels_list,
        "attention_mask": attention_mask_list,
    }
    # Create a Hugging Face Dataset from the dictionary
    dataset = Dataset.from_dict(data_dict)
    # Shuffle the dataset
    dataset = dataset.shuffle(seed=42)
    # Split the dataset into training, validation, and test sets
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))    
    train_dataset = dataset.select(range(train_size))
    val_dataset = dataset.select(range(train_size, train_size + val_size))
    test_dataset = dataset.select(range(train_size + val_size, len(dataset)))   
    
    # original_ds["train"] = original_ds["train"].concatenate(train_dataset)
    # original_ds["validation"] = original_ds["validation"].concatenate(val_dataset)
    # original_ds["test"] = original_ds["test"].concatenate(test_dataset)
    
    original_ds["train"] = concatenate_datasets([original_ds["train"],train_dataset])
   
    original_ds["validation"] = concatenate_datasets( [original_ds["validation"], val_dataset])
    original_ds["test"] = concatenate_datasets([original_ds["test"], test_dataset])
    # Save the updated dataset to disk
    original_ds.save_to_disk(save_path)
    return original_ds

def chunk_2d_list(data, num_chunks):
    # Calculate the size of each chunk
    chunk_size = len(data) // num_chunks
    # Create a list of chunks for each input list
    data_chunk = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
   
    # Return the list of chunks for each input list
    return data_chunk

In [3]:

corpus_type=Type.sents_raw
dataset_path="./built_datasets/all.01/"
data = Loader().load_corpus("peykareh",corpus_type=corpus_type,shuffle_sentences=True)
data += Loader().load_corpus("bijankhan",corpus_type=corpus_type,shuffle_sentences=True)

labeler = Labeler()
# pretrained_model = "HooshvareLab/bert-base-parsbert-uncased"
pretrained_model = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrained_model)


In [4]:
chunked_data = chunk_2d_list(data,20)

# INITIALIZING THE DATASET
input_ids,attention_mask,labels=make_ready_for_ds(chunked_data[0],corpus_type,labeler,tokenizer,512)
build_dataset(input_ids,attention_mask,labels,dataset_path+'0')

input data is in list format, processing it as in a list of sentences


Saving the dataset (0/1 shards):   0%|          | 0/3814 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/476 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/478 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 3814
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 476
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 478
    })
})

In [5]:
for i in range(1,len(chunked_data)):
    print("bijankhan part",i)
    input_ids,attention_mask,labels=make_ready_for_ds(chunked_data[i],corpus_type,labeler,tokenizer,512)
    print(add_data_to_dataset(input_ids,attention_mask,labels,dataset_path+f'{i-1}',dataset_path+f'{i}'))
    

bijankhan part 1
input data is in list format, processing it as in a list of sentences


Saving the dataset (0/1 shards):   0%|          | 0/7666 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/957 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/961 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 7666
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 957
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 961
    })
})
bijankhan part 2
input data is in list format, processing it as in a list of sentences


...

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 76559
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9561
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9589
    })
})
bijankhan part 20
input data is in list format, processing it as in a list of sentences


Saving the dataset (0/2 shards):   0%|          | 0/76559 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9561 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9590 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 76559
    })
    validation: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9561
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 9590
    })
})
